In [3]:
pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.0/1.4 MB 326.8 kB/s eta 0:00:05
   - -------------------------------------- 0.1/1.4 MB 469.7 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.4 MB 476.3 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.4 MB 748.1 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.4 MB 696.3 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 827.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 803.7 kB/s eta 0:00:02
   -------- ------------------------------- 0.3/1.4 MB 819.2 kB/s eta 0:00:02
   ---------- ----------------------------- 0.4/1.4 MB 897.8 kB/s eta 0:00:02
   ---

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [8]:
train_dir = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\train'
test_dir = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\test'

In [10]:
input_shape = (128, 128, 3)
batch_size = 32

In [12]:
data_generator = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 5121 images belonging to 4 classes.


In [16]:
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1279 images belonging to 4 classes.


In [18]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\IT SHOP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features
train_labels = train_generator.classes

C:\Users\IT SHOP\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step


In [22]:
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step


In [24]:
lgb_classifier = lgb.LGBMClassifier(
    boosting_type='gbdt',  # Use gradient boosting
    num_leaves=31,         # Maximum leaves in one tree
    max_depth=-1,          # No maximum depth
    learning_rate=0.1,     # Step size
    n_estimators=100,      # Number of trees
    objective='multiclass',    # multi classification
)
lgb_classifier.fit(train_features, train_labels)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000970 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11774
[LightGBM] [Info] Number of data points in the train set: 5121, number of used features: 52
[LightGBM] [Info] Start training from score -1.966029
[LightGBM] [Info] Start training from score -4.589861
[LightGBM] [Info] Start training from score -0.693342
[LightGBM] [Info] Start training from score -1.050017


LGBMClassifier(objective='multiclass')

In [26]:
test_predictions = lgb_classifier.predict(test_features)

In [28]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 54.73%


In [30]:
model.save('cnn_feature_extractor8.h5')

In [32]:
joblib.dump(lgb_classifier, 'lgb_classifier.pkl')

['lgb_classifier.pkl']

In [34]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor8.h5')
dt_classifier = joblib.load('lgb_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for lightgbm

    # Classify the features with the lightgbm model
    prediction = dt_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\test\NonDemented\26 (86).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
The image is classified as: NonDemented
